# Setting up

In [1]:
!pip install --quiet --upgrade langchain langchain-community langchain-chroma

In [2]:
! pip install -U langchain langchain-openai

  Attempting uninstall: langchain-openai
    Found existing installation: langchain-openai 0.2.5
    Uninstalling langchain-openai-0.2.5:
      Successfully uninstalled langchain-openai-0.2.5


In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4dfde355-cd0a-429a-9a86-69f78aced0bb-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"

In [5]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

USER_AGENT environment variable not set, consider setting it to identify your requests.


'Task Decomposition is a technique where complex tasks are broken down into smaller, more manageable steps. This allows agents or models to think step by step and utilize more computation to tackle difficult tasks effectively. It can be done through simple prompting, task-specific instructions, or with human inputs.'

In [6]:
# cleanup
vectorstore.delete_collection()

In [7]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

len(docs[0].page_content)

43131

In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_path = "https://prontera.ru/eu_prime"
    # header_template = None,
    # verify_ssl = True,
    # proxies = None,
    # continue_on_failure = False,
    # autoset_encoding = True,
    # encoding = None,
    # web_paths = (),
    # requests_per_second = 2,
    # default_parser = "html.parser",
    # requests_kwargs = None,
    # raise_for_status = False,
    # bs_get_text_kwargs = None,
    # bs_kwargs = None,
    # session = None,
    # show_progress = True,
)



In [9]:
!pip install sentence-transformers

In [10]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124


In [11]:
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124


In [12]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("dunzhang/stella_en_400M_v5", trust_remote_code=True).cuda()
# ！The default dimension is 1024, if you need other dimensions, please clone the model and modify `modules.json` to replace `2_Dense_1024` with another dimension, e.g. `2_Dense_256` or `2_Dense_8192` !
# on gpu
# you can also use this model without the features of `use_memory_efficient_attention` and `unpad_inputs`. It can be worked in CPU.
# model = SentenceTransformer(
#     "dunzhang/stella_en_400M_v5",
#     trust_remote_code=True,
#     device="cpu",
#     config_kwargs={"use_memory_efficient_attention": False, "unpad_inputs": False}
# )

c:\Users\mokrota\Documents\GitHub\math_problem_recommender\math_problem_recommender\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
A matching Triton is not available, some optimizations will not be enabled
Traceback (most recent call last):
  File "c:\Users\mokrota\Documents\GitHub\math_problem_recommender\math_problem_recommender\.venv\Lib\site-packages\xformers\__init__.py", line 57, in _is_triton_available
    import triton  # noqa
    ^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'triton'
Some weights of the model checkpoint at dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with anot

In [13]:
# This model supports two prompts: "s2p_query" and "s2s_query" for sentence-to-passage and sentence-to-sentence tasks, respectively.
# They are defined in `config_sentence_transformers.json`
query_prompt_name = "s2p_query"
queries = [
    "print('hello world')",
    "std::cout << \"Hello World!\";",
]

# docs do not need any prompts

docs = [
    "General function in python for printing string output is called print",
    "In c++ to print something out one has to call method std::cout"
]

query_embeddings = model.encode(queries, prompt_name=query_prompt_name)
doc_embeddings = model.encode(docs)
print(query_embeddings.shape, doc_embeddings.shape)

similarities = model.similarity(query_embeddings, doc_embeddings)
print(similarities)
# tensor([[0.8398, 0.2990],
#         [0.3282, 0.8095]])


(2, 1024) (2, 1024)
tensor([[0.5871, 0.5899],
        [0.4614, 0.6874]])


# Scraping latex format of documents

from https://web.evanchen.cc/problems.html

# Splitting scrapped documents by problems

In [14]:
!pip install pylatexenc

In [15]:
import re
from pylatexenc.latexwalker import LatexWalker, LatexEnvironmentNode

def split_latex_by_regex(latex_content, regex_pattern):
    """
    Splits LaTeX content by sections.
    
    Args:
        latex_content (str): The LaTeX document as a string.
    
    Returns:
        dict: A dictionary where each key is a section title, and value is the LaTeX content for that section.
    """
    # Regex to match LaTeX sections
    matches = re.finditer(regex_pattern, latex_content)
    splits = {}
    start_idx = None
    current_section = None

    for match in matches:
        if current_section is not None:
            # Add the previous section
            splits[current_section] = latex_content[start_idx:match.start()].strip()
        current_section = match.group(1)
        start_idx = match.end()
    
    if current_section is not None:
        # Add the last section
        splits[current_section] = latex_content[start_idx:]
    
    return splits

def save_files(files, output_dir="unnamed problems"):
    """
    Saves each LaTeX section to a separate file.
    
    Args:
        sections (dict): Dictionary of sections and their LaTeX content.
        output_dir (str): Directory to save section files.
    """
    import os
    os.makedirs(output_dir, exist_ok=True)

    for file, content in files.items():
        pattern = r"\d+/\d"
        matches = re.findall(pattern, file)
        if matches:
            filename = matches[0].replace('/', '_P') + '.tex'
        else:
            filename = file.replace(' ', '_') + '.tex'

        filename = os.path.join(output_dir, filename)
        with open(filename, "w", encoding="utf-8") as f:
            f.write(content)
        print(f"Saved section '{file}' to {filename}")

if __name__ == "__main__":
    # Example usage
    for i in range(1997, 2002):
        filepath_header = f"IMO-{i}-notes"
        filepath_tail = ".tex"
        with open(filepath_header + filepath_tail, "r", encoding="utf-8") as file:
            latex_content = file.read()

        section_pattern = r"\\section\{(.+?)\}"
        subsection_pattern = r"\\subsection\{(.+?)\}"
        enumerate_item_pattern = r"\\item %% (.+)"

        sections = split_latex_by_regex(latex_content, section_pattern)
        problem_section = sections['Problems']
        problems = split_latex_by_regex(problem_section, enumerate_item_pattern)

        solutions = split_latex_by_regex(latex_content, subsection_pattern)
        save_files(problems, filepath_header), save_files(solutions, filepath_header)


Saved section 'Problem 1' to IMO-1997-notes\Problem_1.tex
Saved section 'Problem 2' to IMO-1997-notes\Problem_2.tex
Saved section 'Problem 3' to IMO-1997-notes\Problem_3.tex
Saved section 'Problem 4' to IMO-1997-notes\Problem_4.tex
Saved section 'Problem 5' to IMO-1997-notes\Problem_5.tex
Saved section 'Problem 6' to IMO-1997-notes\Problem_6.tex
Saved section 'IMO 1997/1' to IMO-1997-notes\1997_P1.tex
Saved section 'IMO 1997/2' to IMO-1997-notes\1997_P2.tex
Saved section 'IMO 1997/3' to IMO-1997-notes\1997_P3.tex
Saved section 'IMO 1997/4' to IMO-1997-notes\1997_P4.tex
Saved section 'IMO 1997/5' to IMO-1997-notes\1997_P5.tex
Saved section 'IMO 1997/6' to IMO-1997-notes\1997_P6.tex
Saved section 'Problem 1' to IMO-1998-notes\Problem_1.tex
Saved section 'Problem 2' to IMO-1998-notes\Problem_2.tex
Saved section 'Problem 3' to IMO-1998-notes\Problem_3.tex
Saved section 'Problem 4' to IMO-1998-notes\Problem_4.tex
Saved section 'Problem 5' to IMO-1998-notes\Problem_5.tex
Saved section 'Probl

In [16]:
def check_similarity(queries, docs):
    query_prompt_name = "s2p_query"

    query_embeddings = model.encode(queries, prompt_name=query_prompt_name)
    doc_embeddings = model.encode(docs)
    print(query_embeddings.shape, doc_embeddings.shape)

    similarities = model.similarity(query_embeddings, doc_embeddings)
    return similarities


In [17]:
!pip install pandas

In [28]:
import pandas as pd
import os

problems = []
for i in range(1997, 2002):
    entry = {}
    entry['year'] = i
    for j in range(1, 7):
        problem_path = os.path.abspath(f'./IMO-{i}-notes/Problem_{j}.tex')
        with open(problem_path, 'r') as f:
            entry[f'problem_{j}'] = f.read()

        solution_path = os.path.abspath(f"./IMO-{i}-notes/{i}_P{j}.tex")
        with open(solution_path, 'r') as f:
            entry[f'solution_{j}'] = f.read()
    problems.append(entry)
    

In [29]:
df = pd.DataFrame(problems)

In [30]:
df

,year,problem_1,solution_1,problem_2,solution_2,problem_3,solution_3,problem_4,solution_4,problem_5,solution_5,problem_6,solution_6
0,1997,In the plane there is an infinite chessboard.\...,\textsl{Available online at \url{https://aops....,Let $ABC$ be a triangle with $\angle A < \min(...,\textsl{Available online at \url{https://aops....,"Let $x_1$, $x_2$, \dots, $x_n$ be real numbers...",\textsl{Available online at \url{https://aops....,An $n \times n$ matrix whose entries come\nfro...,\textsl{Available online at \url{https://aops....,"Find all pairs $(a,b)$ of positive integers sa...",\textsl{Available online at \url{https://aops....,"\nFor each positive integer $n$,\nlet $f(n)$ d...",\n\textsl{Available online at \url{https://aop...
1,1998,A convex quadrilateral $ABCD$ has perpendicula...,\textsl{Available online at \url{https://aops....,"In a competition, there are $a$ contestants\na...",\textsl{Available online at \url{https://aops....,"For any positive integer $n$,\nlet $\tau(n)$ d...",\textsl{Available online at \url{https://aops....,"Determine all pairs $(x,y)$ of positive intege...",\textsl{Available online at \url{https://aops....,Let $I$ be the incenter of triangle $ABC$.\nLe...,\textsl{Available online at \url{https://aops....,\nClassify all functions $f \colon \NN \to \NN...,\n\textsl{Available online at \url{https://aop...
2,1999,A set $S$ of points from the space will be cal...,\textsl{Available online at \url{https://aops....,Find the least constant $C$ such that for any ...,\textsl{Available online at \url{https://aops....,Let $n$ be an even positive integer.\nFind the...,\textsl{Available online at \url{https://aops....,"Find all pairs of positive integers $(x,p)$\ns...",\textsl{Available online at \url{https://aops....,Two circles $\Omega_{1}$ and $\Omega_{2}$ touc...,\textsl{Available online at \url{https://aops....,\nFind all the functions $f \colon \RR \to \RR...,\n\textsl{Available online at \url{https://aop...
3,2000,Two circles $G_1$ and $G_2$ intersect at two p...,\textsl{Available online at \url{https://aops....,"Let $a$, $b$, $c$ be positive real numbers wit...",\textsl{Available online at \url{https://aops....,Let $n \ge 2$ be a positive integer\nand $\lam...,\textsl{Available online at \url{https://aops....,A magician has one hundred cards numbered $1$ ...,\textsl{Available online at \url{https://aops....,Does there exist a positive integer $n$\nsuch ...,\textsl{Available online at \url{https://aops....,"\nLet $\ol{AH_1}$, $\ol{BH_2}$, and $\ol{CH_3}...",\n\textsl{Available online at \url{https://aop...
4,2001,Let $ABC$ be an acute-angled triangle with $O$...,\textsl{Available online at \url{https://aops....,"Let $a$, $b$, $c$ be positive reals. Prove tha...",\textsl{Available online at \url{https://aops....,Twenty-one girls and twenty-one boys took part...,\textsl{Available online at \url{https://aops....,"Let $n > 1$ be an odd integer and let $c_1$, $...",\textsl{Available online at \url{https://aops....,Let $ABC$ be a triangle.\nLet $\ol{AP}$ bisect...,\textsl{Available online at \url{https://aops....,\nLet $a > b > c > d > 0$ be integers satisfyi...,\n\textsl{Available online at \url{https://aop...


In [32]:
problems = []
for index, row in df.iterrows():
    year = row['year']
    for i in range(1, 7):
        problems.append((str(year) + f'P_{i}', row[f"solution_{i}"]))

In [33]:
queries = [i[1] for i in problems]
docs = queries
similarity_tensor = check_similarity(queries, docs)

(30, 1024) (30, 1024)


In [34]:
import torch
score, indeces = torch.topk(similarity_tensor, k=2, dim=1)

In [35]:
other_similar = [(int(el[1]), index, float(score[index][1])) for index, el in enumerate(indeces)]

In [44]:
other_similar

[(19, 0, 0.7538465261459351),
 (18, 1, 0.83721923828125),
 (27, 2, 0.8314324617385864),
 (27, 3, 0.69101881980896),
 (15, 4, 0.8495227098464966),
 (4, 5, 0.82389897108078),
 (28, 6, 0.8359960317611694),
 (26, 7, 0.7299522161483765),
 (15, 8, 0.85323166847229),
 (15, 9, 0.8231666088104248),
 (18, 10, 0.8287205696105957),
 (4, 11, 0.8194943070411682),
 (23, 12, 0.7721055150032043),
 (25, 13, 0.8519243597984314),
 (0, 14, 0.7236874103546143),
 (8, 15, 0.8442881107330322),
 (18, 16, 0.8696157932281494),
 (11, 17, 0.8121345639228821),
 (16, 18, 0.8448580503463745),
 (25, 19, 0.8788169026374817),
 (2, 20, 0.7371670007705688),
 (27, 21, 0.7260762453079224),
 (15, 22, 0.8035176396369934),
 (24, 23, 0.8142293691635132),
 (28, 24, 0.8339641094207764),
 (19, 25, 0.8759431838989258),
 (7, 26, 0.7389252185821533),
 (2, 27, 0.8422297239303589),
 (24, 28, 0.8329799175262451),
 (6, 29, 0.8088075518608093)]

In [41]:
from random import choice

def random_similar(couples, df):
    similar_index, index, score = choice(couples)
    problem = df[index]
    similar_problem = df[similar_index]
    return problem, similar_problem, score, index, similar_index

In [42]:
p, similar_p, score, p_no, similar_p_no = random_similar(other_similar, queries)

In [49]:
def print_problem_no(index):
    print(f"Year {1997 + index // 6}, Problem {index % 6 + 1}")

In [50]:
print_problem_no(p_no)
print_problem_no(similar_p_no)

Year 2001, Problem 4
Year 1997, Problem 3


In [52]:
score

0.8422297239303589

score